# Glacier map with information about selected glaciers

### Import packages

In [ ]:
import panel as pn
import os, numpy as np, pandas as pd, cartopy.crs as ccrs, bokeh
import holoviews as hv, geoviews as gv, datashader as ds
from bokeh.models import ColumnDataSource, OpenURL, TapTool, HoverTool, Plot, Range1d, Text, TapTool, CustomJS, Legend, LegendItem
from bokeh.plotting import figure, show, output_notebook
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import output_file, show, output_notebook, reset_output
from bokeh.models.markers import Circle
hv.extension('bokeh', width=100)

### Function for coordinates calculation

In [ ]:
# prepare coordinates
import math
from ast import literal_eval

def merc(Coords):
    lat = Coords[0]
    lon = Coords[1]

    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale    
    return (x, y)

### Load data

In [ ]:
# a data file with provides a table with RGI IDs and parameters of interest
df_all_all = pd.read_hdf('/home/zora/Dokumente/git/glacier-explorer/data/rgi62_era5_itmix_df.h5', 'df')
df_all_all['rgi_id']= list(df_all_all.axes[0])
df_all = df_all_all[['rgi_id', 'CenLon', 'CenLat', 'Area', 'Zmed','Lmax', 'Surging','Linkages', 'GlacierType','TerminusType', 'IsTidewater']].copy()
df_all.tail()

## Choose glaciers to show in map

In [ ]:
# Fill in the data about the glaciers you want to add. The order of the RGIs has to be increasing. (In all Lists the order of the glaciers has to be the same.):
RGI_No = ['RGI60-01.01104', 'RGI60-01.10689', 'RGI60-03.04539']                 # list of strings (RGI names e.g. ["RGI60-01.01104", "RGI60-01.01105"])
Names = ['Lemon Creek Glacier', 'Columbia Glacier', 'White Glacier']            # list of strings
Glaciertype = ["", "", ""]                                                      # list of strings, optional (Calving / Rock / debris covered / Valley glacier ... )
list_years = [[2911, 2912, 2913, 2914], [411, 412, 413, 414, 415], [1611, 1612, 1613, 1614, 1615]] # list of lists of the years from which mass balance values exist. 
list_mb_values = [[1, 2, 3, 4], [100, 300, 300, 222, 333], [-12, 0, 41, 22, 33]]                   # list of lists of mass balance values in accordance with list of the years

print("The number of the years is equal to the number of mb_values: " + str(len(list_years) == len(list_mb_values)))

In [ ]:
# a new dataframe with the chosen glaciers will be created
df = df_all.loc[df_all['rgi_id'].isin(RGI_No)].copy()
df['name'] = Names
df['Coords'] = list(zip(round(df.CenLat,2), round(df.CenLon,2)))

df['Mb_Years'] = list_years
df['MBvals'] = list_mb_values
df['defined_type'] = Glaciertype
df.tail()

## Build datastructure with all information to use it with bokeh

In [ ]:
# prepare coordinates  
df.loc[:, 'coords_x'] = df['Coords'].apply(lambda x: merc(x)[0])
df.loc[:, 'coords_y'] = df['Coords'].apply(lambda x: merc(x)[1])

source = ColumnDataSource(df)

## Design for map and hover

In [ ]:
# Hover
# parameters shown in the hover: 
TOOLTIPS = """
    <div>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">@name             </span> <br>
            <span style="font-size: 15px;                   ">glacier type:     </span>
            <span style="font-size: 15px;                   ">@defined_type     </span> <br>
            <span style="font-size: 15px;                   ">Length:           </span>
            <span style="font-size: 15px;                   ">@Lmax             </span>
            <span style="font-size: 15px;                   ">m                 </span> <br>
            <span style="font-size: 15px;                   ">@TerminusType     </span> <br>
            <span style="font-size: 15px;                   ">Location          </span>
            <span style="font-size: 10px; color: #696;      ">@CenLat, @CenLon  </span> <br>
        </div>
    </div>
"""
############################################################################
# Create figure for the map
p = figure(x_axis_type="mercator", 
           y_axis_type="mercator",
           plot_width = 1000,
           plot_height = 650,
           background='black',
           tools=['tap', 'wheel_zoom', 'reset', 'pan'])

############################################################################
# Design figure
p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.visible = False
p.ygrid.visible = False
p.xgrid.grid_line_color = None

############################################################################
# add map
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)

# add glacier dots
p.circle(x='coords_x',
         y='coords_y', 
         source=source,
         size=10,
         line_color="black", 
         fill_alpha=0.9,
        )
# add markers around the dots when mouse is directly over glacier dots (belongs to hover)
cr = p.circle(x='coords_x', y='coords_y', source=source, size=20,
                fill_color="white", hover_fill_color="blue",
                fill_alpha=0.0, hover_alpha=0.3,
                line_color=None, hover_line_color="white")
# add hover
hover = HoverTool(tooltips=TOOLTIPS, renderers=[cr])
p.add_tools(hover)

# Mass-balance figure
# build a figure, which shows mass balance values of the glaciers
from bokeh.models.glyphs import Line

plot = figure(plot_width=500, plot_height=500, title="Mass Balance")
plot.xaxis.axis_label = "Years"
plot.xaxis.axis_label_text_font_size = "14pt"
plot.xaxis.major_label_text_font_size = "12pt"
plot.yaxis.axis_label = "[mm w.e./year]"
plot.yaxis.axis_label_text_font_size = "14pt"
plot.yaxis.major_label_text_font_size = "12pt"

lines = plot.line(x = 'x', y = 'y', source = ColumnDataSource({'x': np.arange(5), 'y': [0,0,0,0,0]}), line_width=2)
lines.visible = False
code = '''if (cb_data.source.selected.indices.length > 0){
            lines.visible = true;
            selected_index = cb_data.source.selected.indices[0];
            lines.data_source.data['x'] = years[selected_index]
            lines.data_source.data['y'] = mbvals[selected_index]
            lines.data_source.change.emit(); 
          }'''

# connect with map
p.select(TapTool).callback = CustomJS(args = {'lines': lines, 'mbvals': df['MBvals'], 'years': df['Mb_Years']}, code = code)

## Design Application

In [ ]:
# Panel
title       = '<div style="font-size:38px; color: #326a86; font-weight: bold" >Glacier Gallery</div>'
instruction = '<div style="font-size:15px; color: #326a86" >Select a glacier by clicking on a dot on the world map. The figure on the right will show the corresponding mass balance values.</div>'
oggm_logo   = '<a href="http://edu.oggm.org"><img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/oggm_loupe.png" width=220></a>'
pn_logo     = '<a href="https://panel.pyviz.org"><img src="http://panel.pyviz.org/_static/logo_stacked.png" width=60></a>'

header = pn.Row(pn.Pane(oggm_logo),  pn.layout.Spacer(width=30), 
                pn.Column(pn.Row(pn.Spacer(width=130), pn.Pane(title, width=400)), pn.Pane(instruction, width=1200)),
                pn.layout.HSpacer()
                )
# Put plot and text together and show it
app = (pn.Column(header, pn.Row(pn.Pane(pn_logo, width=100), p, pn.Spacer(width=5), plot), width_policy='max', height_policy='max'))
app.servable()